## Cheat-Sheet

In [3]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#Reading data file
df = pd.read_csv('data3.csv')
#Filtering Data
df = df[df['FACE']  >= 50000]

df = pd.concat([df_train, df_test])

In [ ]:
#Box-Cox transformation
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method = 'box-cox', standardize = False)

# reshaping the column FACE to be in the appropriate dimension for box-cox transformation
FACEbc = np.asarray(df['FACE'])
FACEbc = FACEbc.reshape(-1,1)

# converting type of FACE to float instead of int
# so that I can convert 0 to a very small number instead
FACEbc = FACEbc.astype(float)
FACEbc[FACEbc == 0] = 0.0000000001

pt.fit(FACEbc)

#Adding the transformed FACE values into a new column on dataframe
FACEbc = pt.transform(FACEbc)
df['FACEbc'] = FACEbc

skew1 = round(df['FACE'].skew(),3)
skew2 = round(df['FACEbc'].skew(),3)
lambdas = pt.lambdas_[0]

In [ ]:
#Centered and Normalizing
from numpy.linalg import norm

def center_normalize_df (dataframe):
    
    for column in dataframe:
        dataframe[column] = dataframe[column] - np.mean(dataframe[column])
        dataframe[column] = dataframe[column] / norm(dataframe[column])
    
    return dataframe


def center_normalize_array (array):
    array = array - np.mean(array)
    array = array / norm(array)
    
    return array

In [ ]:
#plotting

#Ploting two axes in one plot
#Plotting Histogram
f, (ax1, ax2) = plt.subplots(1, 2)
ax1.hist(df['FACE'])
ax1.set_title('histogram: skew = %s' % skew1)
ax1.set_xlabel('original')

ax2.hist(df['FACEbc'])
ax2.set_title('histogram: skew = %s' % skew2)
ax2.set_xlabel('optimal transformation: %s = %s' % (chr(0x03BB), round(lambdas,3)))

#plot
plt.show()

#save to file
plt.tight_layout()
plt.savefig("histogram.png")



#plotting line plot
a = np.arange(0,updates-1,1)
b = np.flip(results[4])

plt.plot(a,b)
plt.title('loss')
plt.xlabel('update number')
plt.ylabel('loss')

plt.show()


#Plotting two line with scatter
y_truth = 2 * np.sin(0.5*x - 3) +  0.1*x
y_fitted = B[0,0] * np.sin(B[0,1]*x - B[0,2]) +  B[0,3]*x

plt.scatter(x,y, label = "observations", alpha=0.5)
plt.plot(x,y_truth, label = "truth")
plt.plot(x,y_fitted, label = "fitted")
plt.legend(loc="upper left")
plt.plot
plt.xlabel('x')
plt.ylabel('y')

plt.show()



In [ ]:
#Dummy Variables for categorical variables
#creating dummy variables for the categorical variable MARSTAT 
df_dc = pd.get_dummies(df, columns=['MARSTAT'])
df_dc.head()

#Adding ln(INCOME) as a new column on the dataframe 
df_dc['logINCOME'] = np.log(df_dc['INCOME'])

In [ ]:
#Linear Regression
#Computing linear regression
#Formula FACEbc = B0 + B1*EDUCATION + B2*NUMHH + B3*logINCOME + B4*MARSTAT_0 + B5*MARSTAT_2 + E
from sklearn.linear_model import LinearRegression
X = df_dc[['EDUCATION', 'NUMHH', 'logINCOME', 'MARSTAT_0', 'MARSTAT_2']]
y = df_dc['FACEbc']
reg = LinearRegression().fit(X, y)
B = reg.coef_
B = np.insert(B, 0, reg.intercept_, axis=0)
print(B)

#Computing standard error with n-1 degrees of freedom
SE = np.std(y, ddof=1)
print(SE)

#Computing R2
y_pred = reg.predict(X)
from sklearn.metrics import r2_score
R2 = r2_score(y, y_pred)
print(R2)




#Conducting linear regression using OLS method
import statsmodels.api as sm

X = df_dc[['EDUCATION', 'NUMHH', 'logINCOME', 'MARSTAT_0', 'MARSTAT_2']]
y = df_dc['FACEbc']

X2 = sm.add_constant(X)

lm = sm.OLS(y, X2).fit()
print(lm.summary())




### Prediction

#Conducting prediction with the married man profile as described from the question
X3 = np.array([['16', '4', np.log(120000), '0', '0']])
Y3 = reg.predict(X3)

#Computing upper and lower interval
Y3_upper, Y3_lower = Y3+SE, Y3-SE
#inverse transform after

#Inversing the box-cox transformation for the prediction
Y3 = Y3.reshape(-1,1)
Y3 = pt.inverse_transform(Y3)

#Inversing the box-cox transformation of the standard error
SE = SE.reshape(-1,1)
SE = pt.inverse_transform(SE)


print('95%% likelihood that the true value is between $%.2f and $%.2f' % (Y3_lower, Y3_upper))
print('True value: $%.2f' % Y3)


In [ ]:
#Using lasso to eliminate some features
X = df2[['x1', 'x2', 'x3', 'x4', 'x5']] #without response variable
y = df2['y'] #response

from sklearn.model_selection import KFold
from sklearn.linear_model import LassoCV

#lambdas = np.linspace(0,0.1,101)  # if trying a specific range of lambda

cv = KFold(n_splits=10, shuffle=True, random_state=10)
#model = LassoCV(alphas = lambdas, cv=cv) # if spec range of lambda
model = LassoCV(cv=cv)
lassocv = model.fit(X, y)

B_lasso = lassocv.coef_
B_lasso = np.insert(B_lasso, 0, lassocv.intercept_, axis=0)


print('The Coef are')
print(B_lasso)
#print('\nlambda best is = %f' % lassocv.alpha_)

# transformed data set after lasso
df_lasso =  df2.loc[:, df2.columns != 'y'] * lassocv.coef_
df_lasso = df_lasso.loc[:, (np.abs(df_lasso) > np.abs(0.00)).any(axis=0)]
df_lasso = pd.concat([df_lasso, np.log(df2['y'])], axis = 1) #adding back the response variable
df_lasso

In [ ]:
#table
iteration2 = np.arange(0,iteration+2, 1)
df2 = pd.DataFrame(list(zip(iteration2, B_history, np.flip(loss_history))),
                   columns = ['iteration', 'B_hat', 'loss'])
df2

In [ ]:
#RMSE
RMSE = np.sqrt(((y_hat - y) ** 2).mean())
print('The estimated RMSE via LOOCV is\nRMSE = %s' % (RMSE))

#RMSE using sklearn
from sklearn.metrics import mean_squared_error
mean_squared_error(y, y_hat, squared=False)

In [ ]:
# 10-Fold Cross Validation

def cross_validation (df, func):
    from sklearn.model_selection import KFold
    kf = KFold(n_splits = 10, shuffle = True, random_state = 10)
    kf_rmse = []
    
    for train, test in kf.split(df):
        X_train = df.iloc[train].loc[:, df.columns != 'SalePrice']
        X_train = X_train.squeeze()
        X_test = df.iloc[test].loc[:, df.columns != 'SalePrice']
        y_train = df.iloc[train].loc[:,'SalePrice']
        y_test = df.iloc[test].loc[:,'SalePrice']
        
        reg = func.fit(X_train, y_train)
        y_hat = reg.predict(X_test)
        
        from sklearn.metrics import mean_squared_error
        kf_rmse.append(mean_squared_error(y_test, y_hat, squared=False))
        
    kf_RMSE = (1/10) * np.sum(kf_rmse)
        
    return (kf_RMSE)

In [ ]:
#10 Fold CV
x = df['x']
y = df['y']
B = np.array([0.5, 0.75])
B = B.reshape(1,-1)
#kf_df = pd.DataFrame(zip(x,y), columns = ['x', 'y'])

from sklearn.model_selection import KFold
kf = KFold(n_splits = 10, shuffle = True, random_state = 10)
kf_rmse = []

for train, test in kf.split(df):
    X_train = df.iloc[train].loc[:, df.columns != 'y']
    X_train = X_train.squeeze()
    X_test = df.iloc[test].loc[:, df.columns != 'y']
    y_train = df.iloc[train].loc[:,'y']
    y_test = df.iloc[test].loc[:,'y']

    nr = newton_raphson(X_train,y_train,B,10000)
    
    y_hat = np.exp(nr[1][0,1] + nr[1][0,1]*X_test)
    kf_rmse.append(mean_squared_error(y_test, y_hat, squared=False))

kf_RMSE = (1/10) * np.sum(kf_rmse)




from sklearn.model_selection import KFold
kf = KFold(n_splits = 10, shuffle = True, random_state = 10)
kf_rmse = []
kf_r2 = []

for train, test in kf.split(df2):
    X_train = df2.iloc[train].loc[:, df2.columns != 'y']
    X_train = X_train.squeeze()
    X_test = df2.iloc[test].loc[:, df2.columns != 'y']
    y_train = df2.iloc[train].loc[:,'y']
    y_test = df2.iloc[test].loc[:,'y']

    reg_kf = LinearRegression().fit(X_train, y_train)    
    y_hat = reg_kf.predict(X_test)
    
    kf_rmse.append(mean_squared_error(y_test, y_hat, squared=False))
    kf_r2.append(r2_score(y_test,y_hat))

kf_RMSE = (1/10) * np.sum(kf_rmse)
kf_R2 = (1/10) * np.sum(kf_r2)
print(kf_RMSE)
print(kf_R2)

In [ ]:
def calc_gradient(B, y, x):
    #depends on the dimmension of B
    
    B0 = B[0,0] 
    B1 = B[0,1]
    
    f0 = -2*np.exp(B0+B1*x)*(y-np.exp(B0+B1*x)) # dR/dB_0
    f1 = -2*np.exp(B1*x+B0)*x*(y-np.exp(B0+B1*x)) # dR/dB_1
    
    return np.array([np.sum(f0), np.sum(f1)])

def calc_Jacobian(B, y, x):
    #depends on the dimmension of B
    
    B0 = B[0,0]
    B1 = B[0,1]
    
    df0b0 = -2* (np.exp(B0+B1*x)*y - 2*np.exp(2*B1*x+2*B0)) # df0 / dB_0
    df0b1 = -2* (np.exp(B0+B1*x)*x*y - 2*np.exp(2*B1*x+2*B0)*x) # df0 / dB_1
    df1b0 = -2*x* (np.exp(B1*x+B0)*y - 2*np.exp(2*B1*x+2*B0)) # df1 / dB_0
    df1b1 = -2*x* (np.exp(B1*x+B0)*x*y - 2*np.exp(2*B1*x+2*B0)*x) # df1 / dB_1
    
    return np.array([
            [np.sum(df0b0), np.sum(df0b1)], 
            [np.sum(df1b0), np.sum(df1b1)]
            ])

#Gradient Descent
"""
Loss Function = R(B) = SUM{ R_i(B) } = SUM{ (y_i - yhat_i(B))^2 }

B(r+1) = B(r) - eta * Grad( Loss Function )
B(r+1) = B(r) - eta * SUM{ Grad{ (y_i - yhat_i(B))^2  }}

"""


def gradient_descent(x, y, B, learning_rate, max_iter):
    
    loss_history = []
    iteration = 0
    B_history = [B]
    
    for i in range(max_iter):
        B = B.reshape(1,-1)
    
        B0 = B[0,0] 
        B1 = B[0,1]
        B2 = B[0,2]
        B3 = B[0,3]

        y_hat = B0*np.sin(B1*x-B2)+B3*x  # depends on the model function
        loss = np.sum((y-y_hat)**2)
        
        #stop if there are no improvements in the loss
        if iteration > 0:
            if np.abs(loss - loss_history[0]) == 0:
                iteration = iteration - 1
                break
                
        #keep track of the loss for each iteration)
        loss_history.insert(0,loss)
        
        gradient = calc_gradient(B,y,x)
        
        diff = learning_rate * gradient

        B = B-diff
        B_history.append(B)
        
        iteration = iteration + 1
        
    return loss, learning_rate, B, iteration, loss_history

#Newton Raphson
"""
Loss Function = R(B) = SUM{ R_i(B) } = SUM{ (y_i - yhat_i(B))^2 }

B(r+1) = B(r) - Jac( Loss Function )^-1 * Grad( Loss Function )
B(r+1) = B(r) - Jac( (y_i - yhat_i(B))^2 )^-1 *  Grad( (y_i - yhat_i(B))^2 )

"""


def newton_raphson(x, y, B, max_iter):
    
    loss_history = []
    iteration = 0
    B_history = [B]

        
    for i in range(max_iter):        
        y_hat = np.exp(B[0,0] + B[0,1]*x)
        loss = np.sum((y - y_hat)**2)
                            
        #stop if there are no improvements in the loss
        if iteration > 0:
            if np.abs(loss - loss_history[0]) == 0:
                iteration = iteration - 1
                break   
        
        #keep track of the loss for each iteration
        loss_history.insert(0,loss)
                
        gradient = calc_gradient(B,y,x)
        Jacobian = calc_Jacobian(B,y,x)
        diff = np.matmul(np.linalg.inv(Jacobian),gradient)
        
        B = B - diff
        B_history.append(B)

        iteration = iteration + 1       
        
        
                
    return loss, B, iteration, loss_history, B_history

## Regression

In [1]:
#Linear Regression
from sklearn.linear_model import LinearRegression
model = LinearRegression()
x = df1.loc[:, df1.columns != 'SalePrice']
y = df1['SalePrice']

model.fit(x, y)

y_hat = model.predict(x)

from sklearn.metrics import mean_squared_error
model_train_rmse = mean_squared_error(y, y_hat, squared=False)
#score = model.score(x, y)
print(model_train_rmse)
model_cv_rmse = cross_validation(df1,LinearRegression())

NameError: name 'df1' is not defined